# Unigrams, Bigrams, and Trigrams in Naive Bayes Classifiers

Math of Intelligence Week 6 Challenge - https://www.youtube.com/watch?v=PrkiRVcrxOs&t=7s

In this notebook I will explore the performance of ngram words in a naive bayes classifier. I will look at how they perform across two data sets: 
    1) A Spam SMS dataset 
    2) Rap lines from Biggie Smalls and 2Pac

In [491]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [492]:
df = pd.read_csv('./spamSMS.csv', usecols=[0,1], encoding='latin-1')
df.columns = ['label','body']
# label spam as 1, not spam as 0
df['label'] = df['label'].replace(["ham","spam"],[0,1])
data = df.values

In [493]:
class ngrams_bayes():
    
    def __init__(self, data, n=2, split=0.75):
        
        # split into training and testing data
        self.train_data, self.test_data = train_test_split(data,
                                                          train_size=split)
        # convert into n grams
        self.train_data = [[item[0], self.ngrams(n, item[1])] for item in self.train_data]
        self.test_data = [[item[0], self.ngrams(n, item[1])] for item in self.test_data]
        
        # count unique n grams in training data
        flattened = [gram for message in self.train_data for gram in message[1]]
        self.unique = len(set(flattened))
        
        # init dicts
        self.trainPositive = {}
        self.trainNegative = {}
        # counters
        self.posGramCount = 0
        self.negGramCount = 0
        self.spamCount = 0
        # priors
        self.pA = 0
        self.pNotA = 0
        
    def ngrams(self, n, text):
        text = text.split(' ')
        grams = []
        for i in range(len(text)-n+1):
            gram = ' '.join(text[i:i+n])
            grams.append(gram)
        return grams 
    
    def train(self):
        
        for item in self.train_data:
            label = item[0]
            grams = item[1]
            if label == 1:
                self.spamCount += 1   
            for gram in grams:
                if label == 1:
                    self.trainPositive[gram] = self.trainPositive.get(gram, 0) + 1
                    self.posGramCount += 1
                else:
                    self.trainNegative[gram] = self.trainNegative.get(gram, 0) + 1
                    self.negGramCount += 1
                    
        self.pA = self.spamCount/float(len(self.train_data))
        self.pNotA = 1.0 - self.pA
        
    def classify(self, text, alpha=1.0):
        
        self.alpha = alpha
        isSpam = self.pA * self.conditionalText(text, 1)
        notSpam = self.pNotA * self.conditionalText(text, 0)
        if (isSpam > notSpam):
            return 1
        else:
            return 0
        
    def conditionalText(self, grams, label):
        result = 1.0
        for ngram in grams:
            result *= self.conditionalNgram(ngram, label)
        return result
    
    def conditionalNgram(self, ngram, label):
        alpha = self.alpha
        if label == 1:
            return ((self.trainPositive.get(ngram,0)+alpha) /
                    float(self.posGramCount+alpha*self.unique))
        else:
            return ((self.trainNegative.get(ngram,0)+alpha) /
                    float(self.negGramCount+alpha*self.unique))
            
    def evaluate_test_data(self):
        results = []
        for test in self.test_data:
            label = test[0]
            text = test[1]
            ruling = self.classify(text)
            if ruling == label:
                results.append(1) 
            else:
                results.append(0) 
                
        print("Evaluated {} test cases. {:.2f}% Accuracy".format(len(results), 100.0*sum(results)/float(len(results))))
        return sum(results)/float(len(results))

In [494]:
unigram_bayes = ngrams_bayes(data,1)

In [495]:
unigram_bayes.train()

In [496]:
unigram_bayes.evaluate_test_data()

Evaluated 1393 test cases. 96.48% Accuracy


0.964824120603015

In [497]:
bigram_sms= ngrams_bayes(data,2) 
bigram_sms.train()
bigram_sms.evaluate_test_data()


Evaluated 1393 test cases. 75.45% Accuracy


0.7544867193108399

In [498]:
trigram_sms = ngrams_bayes(data,3) 
trigram_sms.train()
trigram_sms.evaluate_test_data()

Evaluated 1393 test cases. 48.17% Accuracy


0.48169418521177315

Here we can see that our bayesian classifier performs well with unigrams, ok with bigrams, and is basically guessing randomly when it comes to trigrams. Increasing the size of your grams does not help this classifier classify this dataset. This is likely due to the data being made of up of short messages with highly specific and colloquial words. Nearly none of the trigrams will occur more than once in this dataset. I imagine that larger ngrams used in a baysian classifer would work well with something like product reviews which are longer than text messages and use less colloquial language. 

# How well does a Naive Bayesian Classification System perform at distinguishing between lines from Biggie Smalls and 2Pac?
![biggie-pac](./biggie-pac.jpg)
Let's look at how well our classifier performs at telling the difference between East Coast and West Coast 90s hip hop, specifically with The Notorious BIG and 2Pac. 

I gathered this dataset myself by grabbing lyrics of off http://www.azlyrics.com and editing the lyrics text to only include verses by Biggie and 2Pac. In total I collected 32 songs, 16 from each. 

Below I preprocess the data to remove special characters, make everything lowercase, and I split each song up into lines and labeled the lines as either a Biggie line or a Pac line. 

In [515]:
biggie_df = pd.read_csv('./biggie_lyrics.csv', usecols=[1], encoding='latin-1', header=None)
biggie_df.columns = ["lyrics"]
biggie_df["lyrics"] = biggie_df["lyrics"].str.replace('[^\w\s]','')
biggie_df["lyrics"] = biggie_df["lyrics"].str.lower()

In [516]:
biggie_df.tail()

,lyrics
11,relax and take notes while i take tokes of the...
12,good evenin ladies and gentlemen\nhows everybo...
13,who shot ya\nseperate the weak from the obsole...
14,when i die fuck it i wanna go to hell\ncause i...
15,when the lala hits ya lyrics just splits ya\nh...


In [517]:
pac_df = pd.read_csv('./2pac_lyrics.csv', usecols=[1], encoding='latin-1', header=None)
pac_df.columns = ["lyrics"]
pac_df["lyrics"] = pac_df["lyrics"].str.replace('[^\w\s]','')
pac_df["lyrics"] = pac_df["lyrics"].str.lower()

In [518]:
pac_df.head()

,lyrics
0,little something for my godson elijah\nand a l...
1,yo mo bee mayn drop that shit\nyou know what t...
2,rest in peace to my motherfucker biggy smallz\...
3,makaveli in this killuminati\nall through your...
4,its just me against the world\nnothin to lose\...


In [519]:
biggie_lyrics = biggie_df["lyrics"].values
biggie_lyrics = [ song.split('\n') for song in biggie_lyrics]
biggie_lyrics = [line for song in biggie_lyrics for line in song]
pac_lyrics = pac_df["lyrics"].values
pac_lyrics = [ song.split('\n') for song in pac_lyrics]
pac_lyrics = [line for song in pac_lyrics for line in song]

rap_lines = [] 

for line in biggie_lyrics:
    if len(line.split()) > 3:
        rap_lines.append(np.array([0,str(line)]))
        
for line in pac_lyrics:
    if len(line.split()) > 3:
        rap_lines.append(np.array([1,str(line)]))
        
rap_lines = np.array(rap_lines)


In [520]:
rap_lines = pd.DataFrame(rap_lines)
rap_lines.columns = ["label","line"]
rap_lines.head()
rap_lines['label'] = rap_lines['label'].replace(['0','1'],[0,1])

In [521]:
bayes_biggie_vs_pac = ngrams_bayes(rap_lines.values, 1, 0.9)

In [522]:
bayes_biggie_vs_pac.train() 

In [523]:
bayes_biggie_vs_pac.evaluate_test_data()

Evaluated 197 test cases. 75.13% Accuracy


0.751269035532995

Since we have a small data set, let's run multiple trials with different train-test splits to get a better idea of what our average classification accuracy using this method.

In [524]:
results = []

for _ in range(10):
    unigram = ngrams_bayes(rap_lines.values, 1, 0.9)
    unigram.train()
    results.append(unigram.evaluate_test_data())
    
print("Average Accuracy: {:.2f}".format(sum(results)/float(len(results))))

Evaluated 197 test cases. 73.10% Accuracy
Evaluated 197 test cases. 76.14% Accuracy
Evaluated 197 test cases. 75.13% Accuracy
Evaluated 197 test cases. 79.19% Accuracy
Evaluated 197 test cases. 72.08% Accuracy
Evaluated 197 test cases. 71.57% Accuracy
Evaluated 197 test cases. 74.11% Accuracy
Evaluated 197 test cases. 71.57% Accuracy
Evaluated 197 test cases. 70.05% Accuracy
Evaluated 197 test cases. 74.62% Accuracy
Average Accuracy: 0.74


not bad, but how do bigram and trigram compare?

##### Bigram

In [525]:
results = []
for _ in range(10):
    bigram_net = ngrams_bayesnet(rap_lines.values, 2, 0.9)
    bigram_net.train()
    results.append(bigram_net.evaluate_test_data())
    
print("Average Accuracy: {:.2f}".format(sum(results)/float(len(results))))

Evaluated 197 test cases. 68.02% Accuracy
Evaluated 197 test cases. 69.04% Accuracy
Evaluated 197 test cases. 69.54% Accuracy
Evaluated 197 test cases. 73.10% Accuracy
Evaluated 197 test cases. 72.59% Accuracy
Evaluated 197 test cases. 72.08% Accuracy
Evaluated 197 test cases. 74.62% Accuracy
Evaluated 197 test cases. 66.50% Accuracy
Evaluated 197 test cases. 71.57% Accuracy
Evaluated 197 test cases. 64.97% Accuracy
Average Accuracy: 0.70


#### Trigram

In [526]:
results = []
for _ in range(10):
    trigram_net = ngrams_bayesnet(rap_lines.values, 2, 0.9)
    trigram_net.train()
    results.append(trigram_net.evaluate_test_data())
    
print("Average Accuracy: {:.2f}".format(sum(results)/float(len(results))))


Evaluated 197 test cases. 71.07% Accuracy
Evaluated 197 test cases. 71.07% Accuracy
Evaluated 197 test cases. 70.05% Accuracy
Evaluated 197 test cases. 71.57% Accuracy
Evaluated 197 test cases. 70.56% Accuracy
Evaluated 197 test cases. 72.08% Accuracy
Evaluated 197 test cases. 72.08% Accuracy
Evaluated 197 test cases. 64.47% Accuracy
Evaluated 197 test cases. 71.07% Accuracy
Evaluated 197 test cases. 64.47% Accuracy
Average Accuracy: 0.70



Unigrams seem to have a slight edge on bigrams and trigrams but using trigrams doesn't yeild horrible results on this rap data like it did when classing sms messages. My guess is that, while these raps contain highly colloquial words, there are phrases unique to Biggie and Pac that they use repeatedly while with sms messages the sequence of words is more arbitrary. 